In [8]:
%load_ext autoreload
%autoreload 2

In [10]:
import sys
sys.path.append('..')

In [18]:
import os

In [14]:
from dataset import *

In [15]:
VoiceDataset

dataset.VoiceDataset

In [64]:
mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=16000,
        n_fft=1024,
        hop_length=512,
        n_mels=64
    )
dataset = VoiceDataset('../data', mel_spectrogram, 16000,)

In [65]:
len(dataset)

5718

In [66]:
dataset[0]

(tensor([[[0.2647, 0.0247, 0.0324,  ..., 0.0000, 0.0000, 0.0000],
          [0.0812, 0.0178, 0.0890,  ..., 0.0000, 0.0000, 0.0000],
          [0.0052, 0.0212, 0.1341,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.5154, 0.3950, 0.4497,  ..., 0.0000, 0.0000, 0.0000],
          [0.1919, 0.4804, 0.5144,  ..., 0.0000, 0.0000, 0.0000],
          [0.1208, 0.4357, 0.4016,  ..., 0.0000, 0.0000, 0.0000]]]),
 'aman')

In [67]:
dataset[0][0].shape

torch.Size([1, 64, 313])